In [10]:
from random import randint
import itertools
import time
import pickle
import numpy as np
from nomadic_trucker_new import nomadictrucker

In [17]:
def get_benchmark_action(state):
    truck_location = model['get_state_exp'](state)[0]
    load = model['get_state_exp'](state)[3]

    if all(load_item == 0 for load_item in load): # if everything in the load vector is 0, stay at current location
        predicted_optimal_action = truck_location

    else:
        distances = []
        indices = []
        for i in range(len(load)):
            if load[i] == 1:
                distance = model['get_distance'](truck_location, i)
                distances.append(distance)
                indices.append(i)

        furthest_distance = max(distances)
        furthest_distance_index = indices[distances.index(furthest_distance)]
        predicted_optimal_action = furthest_distance_index # go to the location furthest away that needs a load delivered

    return predicted_optimal_action

In [21]:
def Benchmark_Simulation(model, pi_s, load_list):
    total_reward = 0
    next_state = 0
    for i in range(1000):
        #print('initial state',model['get_state_exp'](next_state))
        predicted_optimal_action = get_benchmark_action(next_state)
        #print('predicted optimal action:',predicted_optimal_action)
        reward = model['get_rewards'](next_state, predicted_optimal_action)
        #print('reward is:', reward)
        total_reward += reward
        truck_location, day, trailer, load = model['get_actual_next_grid_state'](predicted_optimal_action, (next_state))
        load = load_list[i]
        next_state = model['get_numeric_state_nomadic_trucker'](truck_location, day, trailer, load) # move to next state
    return total_reward

In [24]:
# possible states
s = [2, 3]

# possible days
d = [1, 3, 7]

# possible trailer types
t = [1, 3, 3]

In [25]:
for size in s:
    for days, trailers in zip(d,t):
        model = nomadictrucker(size,days,trailers)
        pi_s = pickle.load(open("Benchmark"+str(len(model['states']))+".pickle","rb"))
        load_list = pickle.load(open("Load_"+str(size)+"_"+str(days)+".pickle", "rb"))
        Result = Benchmark_Simulation(model, pi_s, load_list)
        print("Result for",str(len(model['states'])), "states:", Result)

Result for 64 states: 141511.68849500018
Result for 576 states: 227752.41226500002
Result for 1344 states: 207324.981055
Result for 4608 states: 346903.63302666694
Result for 41472 states: 104213.54595333328
Result for 96768 states: 104213.54595333328
